In [1]:
import joblib

# Load the SVC model
loaded_svc_model = joblib.load('models/svc_model.pkl')

# Load the OneClassSVM model
loaded_one_class_svm = joblib.load('models/one_class_svm_model.pkl')

In [2]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

2025-02-20 17:11:06.903129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740067866.926496  140746 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740067866.938509  140746 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 17:11:06.970868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [4]:
from keras_facenet import FaceNet
embedder = FaceNet()

def get_embedding(model, face_pixels):
	face_pixels = face_pixels.astype('float32') # 3D (160x160x3)
	face_img = np.expand_dims(face_pixels, axis=0)
  # 4D (Nonex160x160x3)
	embedding = model.embeddings(face_img)
	return embedding[0] # 512 D image (1x1x512)

2025-02-20 17:11:11.713683: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
ESP32_STREAM_URL = "http://172.20.10.6:81/stream"  # Change this to your ESP32-CAM IP
SAVE_PATH = "esp32_output.avi"  # Video file name

import threading
import time
import psutil
# Resource monitoring function
def monitor_resources(interval=1):
    process = psutil.Process()
    while True:
        cpu_usage = process.cpu_percent(interval=interval)
        memory_info = process.memory_info()
        memory_usage_mb = memory_info.rss / (1024 * 1024)

        print(f"CPU Usage: {cpu_usage}% | Memory Usage: {memory_usage_mb:.2f} MB")
        print("-" * 30)

        time.sleep(interval)

# Start monitoring in a separate thread
monitor_thread = threading.Thread(target=monitor_resources, daemon=True)
monitor_thread.start()

# Open video stream
cap = cv2.VideoCapture(ESP32_STREAM_URL)

detector = MTCNN()

# Get frame sizeq
ret, frame = cap.read()
if not ret:
    print("⚠️ Failed to read video stream. Check ESP32-CAM.")
    cap.release()
    exit()

frame_height, frame_width, _ = frame.shape
person_ids = {
    1: "Annalise Keating",
    2: "Manuelle"
}

# Set up video writer (MJPEG, 10 FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(SAVE_PATH, fourcc, 10, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Failed to read video stream. Check ESP32-CAM.")
        cap.release()
        exit()

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(img)

    if len(faces) > 0:
        x, y, w, h = faces[0]['box']
        face_img = img[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (160, 160))
        embedding = get_embedding(embedder, face_img)
        pred_probs = loaded_svc_model.predict_proba([embedding])
        pred_label = loaded_svc_model.predict([embedding])[0]
        pred_prob = np.max(pred_probs)
        #print(pred_label)
        cls = f"Predicted Class: {person_ids.get(pred_label, 'Unknown')}, {pred_prob}"
        cv2.putText(frame, cls, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    else:
        print("No face detected")

    cv2.imshow("ESP32-CAM Stream", frame)
    out.write(frame)  # Save frame to video file

    if cv2.waitKey(1) & 0xFF == ord("q"):  # Press 'q' to stop
        break

CPU Usage: 115.9% | Memory Usage: 988.35 MB
------------------------------
CPU Usage: 99.9% | Memory Usage: 782.88 MB
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
CPU Usage: 124.8% | Memory Usage: 1043.97 MB
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
No face detected
CPU Usage: 149.9% | Memory Usage: 979.02 MB
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
CPU Usage: 159.7% | Memory Usage: 1143.64 MB
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
CPU Usage: 145.9% | Memory Usage: 1084.51 MB
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
CPU Usage: 138.8% | Memory Usage: 1114.93 MB
------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step


In [6]:
# Load the model
LRCN_model = load_model('models/LRCN_model___Date_Time_2025_02_20__13_50_05___Loss_0.11162082850933075___Accuracy_1.0.h5')
# Verify the model architecture
LRCN_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ time_distributed                │ (None, 20, 64, 64, 16) │           448 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 20, 16, 16, 16) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 20, 16, 16, 16) │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 20, 16, 16, 32) │         4,640 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 20, 4, 4, 32)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 20, 4, 4, 32)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 20, 4, 4, 64)   │        18,496 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 20, 2, 2, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_8              │ (None, 20, 2, 2, 64)   │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_9              │ (None, 20, 256)        │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,644 (236.89 KB)

 Trainable params: 60,642 (236.88 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [7]:
from collections import deque

ESP32_STREAM_URL = "http://172.20.10.6:81/stream"  # Change this to your ESP32-CAM IP
SAVE_PATH = "esp32_output.avi"  # Video file name

# Open video stream
cap = cv2.VideoCapture(ESP32_STREAM_URL)

# Get frame sizeq
ret, frame = cap.read()
if not ret:
    print("⚠️ Failed to read video stream. Check ESP32-CAM.")
    cap.release()
    exit()

frame_height, frame_width, _ = frame.shape
SEQUENCE_LENGTH = 20
CLASSES_LIST = ['open', 'close']
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64

# Set up video writer (MJPEG, 10 FPS)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(SAVE_PATH, fourcc, 10, (frame_width, frame_height))
predicted_class_name = ''
frames_queue = deque(maxlen=SEQUENCE_LENGTH)
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        print("⚠️ Failed to read video stream. Check ESP32-CAM.")
        cap.release()
        exit()

    resized_frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
    normalized_frame = resized_frame / 255
    frames_queue.append(normalized_frame)

    if len(frames_queue) == SEQUENCE_LENGTH:
        predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_queue, axis=0))[0]
        predicted_label = np.argmax(predicted_labels_probabilities)
        predicted_class_name = CLASSES_LIST[predicted_label] + " : " + str(round(predicted_labels_probabilities[predicted_label] * 100, 2))
    cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    frame_count += 1

    cv2.imshow("ESP32-CAM Stream", frame)
    out.write(frame)  # Save frame to video file

    if cv2.waitKey(10) & 0xFF == ord("q"):  # Press 'q' to stop
        break

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━